# Transfer Learning com MobileNetV2

Neste notebook, implementamos o processo de Transfer Learning utilizando o modelo pré-treinado MobileNetV2. Vamos usar o modelo para classificar imagens de gatos e cães, aproveitando o conhecimento pré-existente do modelo sobre o conjunto de dados ImageNet.

## Etapas do processo:
1. Carregamento do dataset.
2. Preparação dos datasets de treino e validação.
3. Aumentação de dados para evitar overfitting.
4. Criação do modelo base (MobileNetV2).
5. Adição da camada de classificação personalizada.
6. Treinamento e avaliação do modelo.
7. Fine-tuning para melhorar a performance.



In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Transfer Learning com MobileNetV2"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "import matplotlib.pyplot as plt\n",
        "import numpy as np\n",
        "import os\n",
        "import tensorflow as tf\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Download e extração do dataset"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'\n",
        "path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)\n",
        "PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')\n",
        "\n",
        "train_dir = os.path.join(PATH, 'train')\n",
        "validation_dir = os.path.join(PATH, 'validation')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Preparando os datasets de treino e validação"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "BATCH_SIZE = 32\n",
        "IMG_SIZE = (160, 160)\n",
        "\n",
        "train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,\n",
        "                                                            shuffle=True,\n",
        "                                                            batch_size=BATCH_SIZE,\n",
        "                                                            image_size=IMG_SIZE)\n",
        "\n",
        "validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,\n",
        "                                                                 shuffle=True,\n",
        "                                                                 batch_size=BATCH_SIZE,\n",
        "                                                                 image_size=IMG_SIZE)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Aumentação de dados para evitar overfitting"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "data_augmentation = tf.keras.Sequential([\n",
        "    tf.keras.layers.RandomFlip('horizontal'),\n",
        "    tf.keras.layers.RandomRotation(0.2),\n",
        "])\n",
        "\n",
        "preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Criando o modelo base (MobileNetV2)"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "IMG_SHAPE = IMG_SIZE + (3,)\n",
        "base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,\n",
        "                                               include_top=False,\n",
        "                                               weights='imagenet')\n",
        "\n",
        "base_model.trainable = False"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Adicionando a cabeça de classificação"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "global_average_layer = tf.keras.layers.GlobalAveragePooling2D()\n",
        "prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')\n",
        "\n",
        "inputs = tf.keras.Input(shape=(160, 160, 3))\n",
        "x = data_augmentation(inputs)\n",
        "x = preprocess_input(x)\n",
        "x = base_model(x, training=False)\n",
        "x = global_average_layer(x)\n",
        "x = tf.keras.layers.Dropout(0.2)(x)\n",
        "outputs = prediction_layer(x)\n",
        "model = tf.keras.Model(inputs, outputs)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Compilando e treinando o modelo"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "base_learning_rate = 0.0001\n",
        "model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),\n",
        "              loss=tf.keras.losses.BinaryCrossentropy(),\n",
        "              metrics=['accuracy'])\n",
        "\n",
        "initial_epochs = 10\n",
        "\n",
        "history = model.fit(train_dataset,\n",
        "                    epochs=initial_epochs,\n",
        "                    validation_data=validation_dataset)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Avaliando o desempenho do modelo"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "acc = history.history['accuracy']\n",
        "val_acc = history.history['val_accuracy']\n",
        "loss = history.history['loss']\n",
        "val_loss = history.history['val_loss']\n",
        "\n",
        "plt.figure(figsize=(8, 8))\n",
        "plt.subplot(2, 1, 1)\n",
        "plt.plot(acc, label='Training Accuracy')\n",
        "plt.plot(val_acc, label='Validation Accuracy')\n",
        "plt.legend(loc='lower right')\n",
        "plt.ylabel('Accuracy')\n",
        "plt.ylim([min(plt.ylim()),1])\n",
        "plt.title('Training and Validation Accuracy')\n",
        "\n",
        "plt.subplot(2, 1, 2)\n",
        "plt.plot(loss, label='Training Loss')\n",
        "plt.plot(val_loss, label='Validation Loss')\n",
        "plt.legend(loc='upper right')\n",
        "plt.ylabel('Cross Entropy')\n",
        "plt.ylim([0,1.0])\n",
        "plt.title('Training and Validation Loss')\n",
        "plt.xlabel('epoch')\n",
        "plt.show()"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 4
}
